# Exploratory Data Analysis

## Setup
Import the relevant packages.


In [71]:
import ast
import pandas as pd

We have a single file `20220310.csv`. Let's first look at what type of data we 
are dealing with. 

In [72]:
# load in csv as dataframes
# lineterminator option is used as there are some line separation issues.
raw_data = pd.read_csv("raw-data/20220310.csv", lineterminator="\n")

# get summary information on the data frame, including columns and data types
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  5000 non-null   bool   
 1   backdrop_path          4874 non-null   object 
 2   belongs_to_collection  1742 non-null   object 
 3   budget                 5000 non-null   int64  
 4   genres                 5000 non-null   object 
 5   homepage               2418 non-null   object 
 6   id                     5000 non-null   int64  
 7   imdb_id                4938 non-null   object 
 8   original_language      5000 non-null   object 
 9   original_title         5000 non-null   object 
 10  overview               4963 non-null   object 
 11  popularity             5000 non-null   float64
 12  poster_path            4995 non-null   object 
 13  production_companies   5000 non-null   object 
 14  production_countries   5000 non-null   object 
 15  rele

Let's also look at the first few rows to see what we are dealing with. 

In [73]:
raw_data.head()
raw_data.iloc[[0]].transpose()

,0
adult,False
backdrop_path,/iQFcwSGbZXMkeyKrxbPnwnRo5fl.jpg
belongs_to_collection,"{'id': 531241, 'name': 'Spider-Man (Avengers) ..."
budget,200000000
genres,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
homepage,https://www.spidermannowayhome.movie
id,634649
imdb_id,tt10872600
original_language,en
original_title,Spider-Man: No Way Home


## Data Cleaning

First we remove columns that we will obviously not be using, e.g. `backdrop_path`,
`homepage`, etc. 

In [74]:
useless_columns = ["backdrop_path", "belongs_to_collection", "homepage", 
"poster_path", "video"]

raw_data.drop(useless_columns, axis=1, inplace=True)
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 5000 non-null   bool   
 1   budget                5000 non-null   int64  
 2   genres                5000 non-null   object 
 3   id                    5000 non-null   int64  
 4   imdb_id               4938 non-null   object 
 5   original_language     5000 non-null   object 
 6   original_title        5000 non-null   object 
 7   overview              4963 non-null   object 
 8   popularity            5000 non-null   float64
 9   production_companies  5000 non-null   object 
 10  production_countries  5000 non-null   object 
 11  release_date          4980 non-null   object 
 12  revenue               5000 non-null   int64  
 13  runtime               4999 non-null   float64
 14  spoken_languages      5000 non-null   object 
 15  status               

Next we parse `release_date` as a column of date objects, and convert `status` into a Boolean column (`True` for released, `False` otherwise). 

In [75]:
# parse dates
raw_data["release_date"] = pd.to_datetime(raw_data["release_date"])
# convert status column to Boolean, and only keep released films
raw_data["status"] = raw_data["status"].map({"Released": True})

The columns `genres`, `production_companies`, `production_countries`, `spoken_languages`,
`credits`, and `keywords` are lists of quasi-JSON style strings. However, single
quotes have been used, so instead we store them in the dataframe as lists of 
dictionaries.

In [76]:
# parse columns with nested JSON objects as actual JSON objects instead of str
json_cols = [
    "genres", "production_companies", "production_countries", "spoken_languages", 
    "credits", "keywords"
]

for col in json_cols:
    raw_data[col] = raw_data[col].apply(ast.literal_eval)

raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 5000 non-null   bool          
 1   budget                5000 non-null   int64         
 2   genres                5000 non-null   object        
 3   id                    5000 non-null   int64         
 4   imdb_id               4938 non-null   object        
 5   original_language     5000 non-null   object        
 6   original_title        5000 non-null   object        
 7   overview              4963 non-null   object        
 8   popularity            5000 non-null   float64       
 9   production_companies  5000 non-null   object        
 10  production_countries  5000 non-null   object        
 11  release_date          4980 non-null   datetime64[ns]
 12  revenue               5000 non-null   int64         
 13  runtime           

Due to the use of `ast.literal_eval`, we have ended up with the following columns:

- `genres`, `production_companies`, `production_countries`, `spoken_languages` are lists
- `credits` and `keywords` are dictionaries.

Morevover, credits has keys `cast` and `crew`, while keywords simply has one 
key `keywords`. The corresponding values are in the same list of dictionaries structure 
as above. Hence, the best way to clean this data is to create two new columns 
for `cast` and `crew`, and simply reassign the `keywords` column to the actual 
value inside `keywords`. 

In [77]:
list_cols = ["genres", "production_companies", "production_countries", "spoken_languages",
                "cast", "crew", "keywords"]

cast = []
crew = []
for movie in raw_data["credits"]:
    cast.append(movie["cast"])
    crew.append(movie["crew"])
raw_data["cast"] = pd.Series(cast)
raw_data["crew"] = pd.Series(crew)

new_keywords = []
for movie in raw_data["keywords"]:
    new_keywords.append(movie["keywords"])
raw_data["keywords"] = pd.Series(new_keywords)

def helper(col):
    new_col = []
    for val in col:
        new_val = []
        for x in val:
            new_val.append(dict(x))
        
        new_col.append(new_val)
    
    return pd.Series(new_col)

for col in list_cols:
    raw_data[col] = helper(raw_data[col])
    print(col + " done")

genres done
production_companies done
production_countries done
spoken_languages done
cast done
crew done
keywords done


We can now drop the `credits` column. 

In [ ]:
raw_data.drop("credits", axis=1, inplace=True)
raw_data.head()

We finally have lists of valid JSON strings inside the columns. We are now ready to flatten the data and perform one-hot encoding. Whether or not a column should be flattened or one-hot encoded depends on the structure of the model being used. Hence, we define functions to perform either for future use. 

In [ ]:
# creates a flat dataframe based on any column
def flatten(df: pd.DataFrame, col: str):
    """
    INPUTS:
        df:     (pd.DataFrame) a dataframe
        col:    (str) the name of a column in df
    OUTPUTS:
                (pd.DataFrame) a dataframe flattened according to col
    """

    ls_of_temps = []
    for i in range(0, df.shape[0], 1):
        temp_df = pd.json_normalize(df[col][i])
        temp_df["movie_id"] = df["id"][i]
        temp_df["movie_title"] = df["title"][i]

        temp_df.columns = [col + "_tmdb_" + x if x == "id" else x for x in temp_df.columns]
        temp_df.columns = [col + "_" + x if x == "name" else x for x in temp_df.columns]
        
        ls_of_temps.append(temp_df)
    
    return pd.concat(ls_of_temps)

# creates a new one-hot encoded dataframe based on any column
def one_hot(df: pd.DataFrame, col: str):
    """
    INPUTS:
        df:     (pd.DataFrame) a dataframe
        col:    (str) the name of a column in df
    OUTPUTS:
                (pd.DataFrame) a dataframe one-hot encoded according to col
    """
    # flatten data on col
    flat_df = flatten(df, col)
    
    # compute one hot encoding using str.get_dummies
    return flat_df.set_index("movie_id")[col + "_name"].str.get_dummies().groupby(level=0).sum()
    

In [78]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 5000 non-null   bool          
 1   budget                5000 non-null   int64         
 2   genres                5000 non-null   object        
 3   id                    5000 non-null   int64         
 4   imdb_id               4938 non-null   object        
 5   original_language     5000 non-null   object        
 6   original_title        5000 non-null   object        
 7   overview              4963 non-null   object        
 8   popularity            5000 non-null   float64       
 9   production_companies  5000 non-null   object        
 10  production_countries  5000 non-null   object        
 11  release_date          4980 non-null   datetime64[ns]
 12  revenue               5000 non-null   int64         
 13  runtime           

In [83]:
# raw_data["original_language"].unique()
# non-adult films only
clean_data = raw_data[raw_data["adult"] == False]
# released films only 
clean_data = clean_data[clean_data["status"] == True]
# English-language films only
clean_data = clean_data[clean_data["original_language"] == "en"]

clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3869 entries, 0 to 4999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   adult                 3869 non-null   bool          
 1   budget                3869 non-null   int64         
 2   genres                3869 non-null   object        
 3   id                    3869 non-null   int64         
 4   imdb_id               3847 non-null   object        
 5   original_language     3869 non-null   object        
 6   original_title        3869 non-null   object        
 7   overview              3867 non-null   object        
 8   popularity            3869 non-null   float64       
 9   production_companies  3869 non-null   object        
 10  production_countries  3869 non-null   object        
 11  release_date          3867 non-null   datetime64[ns]
 12  revenue               3869 non-null   int64         
 13  runtime           